# Visualisation & Simulation Examples 

You will need the to install the Python libraries [ase](https://wiki.fysik.dtu.dk/ase/) and [nglview](https://github.com/nglviewer/nglview)

e.g.: `pip install ase nglview`

**Note**: Its not expected that you understand in detail what is happening in this notebook yet. Its merely a demonstration of what can be done with a few lines of Python code.  
However, you will understand it at the end of the lecture!

### Visualisation of a Protein

https://www.rcsb.org/structure/2HHB

Hemoglobin is essential for oxygen transport in blood, binding oxygen in the lungs and releasing it in tissues.


In [ ]:
!pip install ase nglview

In [ ]:
import nglview
nglview.show_pdbid("2HHB")  # load "2HHB" from RCSB PDB

###  Molecular Dynamics Simulation of Acetonitrile


A [molecular dynamics](https://en.wikipedia.org/wiki/Molecular_dynamics#:~:text=Molecular%20dynamics%20(MD)%20is%20a,%22evolution%22%20of%20the%20system.) (MD) simulates atomic and molecular movements over time by solving Newton's equations of motion, using force fields to calculate interactions.   
It is widely used in chemical physics, materials science, and biophysics.


In [ ]:
# load the necessary functions and modules
import ase.units as units
import numpy as np
from ase.calculators.acn import ACN
from ase.constraints import FixLinearTriatomic
from ase.io import Trajectory,write, read
from ase.md import Langevin
from ase.visualize import view

# helper functions for showing output of simulation
def show_system(atoms):
    v = view(atoms, viewer="ngl")
    v.view._remote_call("setSize", target="Widget", args=["700px","700px"])
    v.view.add_ball_and_stick()
    v.view.background="black"
    return v

def print_energy(atoms):
    epot = atoms.get_potential_energy()
    ekin = atoms.get_kinetic_energy()
    temperature = atoms.get_kinetic_energy()/len(atoms)/(1.5*units.kB)
    print(f"Potential Energy: {epot:.3f} Kinetic Energy: {ekin:.3f} Temperature: {temperature:.1f}")


In [ ]:
#!cat acn.xyz

In [ ]:
%%time
# read coorrdinates from file
atoms = read("../data/acn.xyz") 
tag = 'acn'

# set up some constraints, to accelerate dynamics
atoms.constraints = FixLinearTriatomic(
    triples=[(3 * i, 3 * i + 1, 3 * i + 2)
             for i in range(27)])

# define the molecular dynamics physics / rules / model
atoms.calc = ACN(rc=np.min(np.diag(atoms.cell)) / 2)
# define the simulation details, temperature, time steps, parameters, logfile name
md = Langevin(atoms, 1 * units.fs,
              temperature_K=300,
              friction=0.01,
              logfile=tag + '.log')

# define a file to save the generated structures during the dynamics
traj = Trajectory(tag + '.traj', 'w', atoms)
md.attach(traj.write, interval=10)
md.attach(print_energy,interval=10,atoms=atoms)

# start the simulation for n time steps
n_steps = 2000
md.run(n_steps)

In [ ]:
# visualization of the dynamics
traj = Trajectory("acn.traj")
show_system(traj)

### Exercise

* Find the temperature parameter. Set the temperature to 100 K and observe the result.
* The result will not be what you expect. Which parameter should be changed in addition?
* How would you change the code to continue on the previous results?